In [52]:
import pandas as pd
import re
import unidecode
import demoji
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
import spacy
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import tokenize
import nltk

%matplotlib inline

nlp = spacy.load("pt")
nlp.Defaults.stop_words |= {"to","uol","mi","budddhetg","the", "ne", "vou", "ta", "via","ex", "pq", "vc","aa","pra","to", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"}

In [53]:
palavras_irrelevantes = set(stopwords.words('portuguese') + list(punctuation))

In [54]:
data = pd.read_csv("data/tweeterLeo/dataTweeter.csv", sep=";", encoding='utf8')

In [55]:
def cleanText(tweet):
    if "|" in tweet:
        tweet = tweet.split("|")[1]
        
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([0-9])", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\_\|\.\,\"\'\!\?\:\;\$\-\(\)\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    le = list(demoji.findall(tweet))
    for i in le:
        tweet = tweet.replace(i, "")
    
    if tweet.startswith('rt '):
        tweet = tweet.replace("rt ", "")
    
    lNewTweet = []
    for i in tweet.split(" "):
        if i not in palavras_irrelevantes and i not in STOP_WORDS:
            lNewTweet.append(i)
    
    newTweet = " ".join(lNewTweet)
        
    return unidecode.unidecode(newTweet.replace(" rt ", ""))

<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \_
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \_
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \_
<ipython-input-55-8df616e6d7d3>:6: DeprecationWarning: invalid escape sequence \w
  tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
<ipython-input-55-8df616e6d7d3>:7: DeprecationWarning: invalid escape sequence \_
  tweet = ' '.join(re.sub("[\_\|\.\,\"\'\!\?\:\;\$\-\(\)\=]", " ", tweet).split())


In [56]:
data["tweet_text_clean"] = data.text.apply(cleanText)

In [57]:
data.head()

,id,user,topic,text,tweet_text_clean
0,"1,31497377565871E+018",DivaDepressao,NaN,RT @PaivaPaulo8: @DivaDepressao pelo amor de D...,amor deus analisa festa
1,"1,31497371872942E+018",DivaDepressao,NaN,RT @andrecarboni666: meu vício agora é pegar ô...,vicio onibus ouvindo filhos gravida
2,"1,31497369440248E+018",DivaDepressao,NaN,RT @bii_aa: Alô dona @NetflixBrasil a senhora ...,alo dona senhora assistiu video viu hahahahhah...
3,"1,3149736672863E+018",DivaDepressao,NaN,@SandroArturBel1 @JornalOGlobo Ihhhhhh,ihhhhhh
4,"1,31497360792013E+018",DivaDepressao,NaN,RT @fatbarbiemarley: @FilhosDaGravida o álcool...,alcool planta edu fala arnica


In [46]:
# todas_palavras = ' '.join([texto for texto in data["tweet_text_clean"]])

# token_espaco = tokenize.WhitespaceTokenizer()
# token_frase = token_espaco.tokenize(todas_palavras)

# token_frase_2 = []

# for i in token_frase:
#     a = cleanText(i)
#     if a:
#         token_frase_2.append(cleanText(i))


In [47]:
# frequencia = nltk.FreqDist(token_frase_2)
# freqmc = frequencia.most_common(10000)

# freqmc

# data[data["user"] == "folha"]["tweet_text_clean"]

In [58]:
#Test Gemsin bigrams
# token_frase_2
# lista_texto = [texto[0] for texto in token_frase_2]
# lista_texto
# # # Build the bigram and trigram models
# t = data["tweet_text_clean"].tolist()
# t

token_ = [doc.split(" ") for doc in data[data["user"] == "UOLNoticias"]["tweet_text_clean"].tolist()]
# # token_
# # token_ = [todas_palavras]
import gensim
from gensim.models.phrases import Phraser

bigram = gensim.models.Phrases(token_, min_count=1, threshold=0.1) # higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[token_], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

l = [trigram_mod[bigram_mod[doc]] for doc in token_]
# l

In [59]:
# dir(bigram_mod)
# # l[6000:]
import gensim.corpora as corpora

In [60]:
# list(id2word.values())
id2word = corpora.Dictionary(l)
corpus = [id2word.doc2bow(text) for text in l]


In [61]:

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [62]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"bolsonaro" + 0.007*"queimadas" + 0.005*"pandemia" + 0.004*"onu" + '
  '0.004*"volta_aulas" + 0.004*"brasil" + 0.003*"rio" + 0.003*"opiniao" + '
  '0.003*"trump" + 0.003*"sp"'),
 (1,
  '0.011*"covid" + 0.009*"bolsonaro" + 0.005*"sp" + 0.004*"brasil" + '
  '0.004*"presidente" + 0.004*"coronavirus" + 0.004*"paulo" + 0.003*"governo" '
  '+ 0.003*"estudo" + 0.003*"opiniao"'),
 (2,
  '0.006*"brasil" + 0.005*"milhoes" + 0.005*"policia" + 0.004*"anos" + '
  '0.004*"eua" + 0.004*"morre" + 0.003*"aponta" + 0.003*"cafe_mercado" + '
  '0.003*"suspeito" + 0.003*"ano"'),
 (3,
  '0.008*"bolsonaro" + 0.004*"opiniao" + 0.003*"mae" + 0.003*"renda_brasil" + '
  '0.003*"saude" + 0.003*"rio" + 0.003*"governo" + 0.003*"bolsa_familia" + '
  '0.003*"guedes" + 0.003*"russomanno"'),
 (4,
  '0.065*"pegou_carona_vexame_guedes" + 0.065*"opiniao_decisao_bolsonaro_maia" '
  '+ 0.004*"sp" + 0.003*"coronavirus" + 0.003*"anos" + 0.003*"mercado" + '
  '0.002*"eleicao" + 0.002*"bolsonaro" + 0.002*"boulos" 

In [63]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=l, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6537048275821594


In [64]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=l, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [65]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [1:22:42<00:00,  9.19s/it]


In [69]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.31,
                                           eta=0.01)

In [70]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=l, dictionary=id2word, coherence='c_v')

In [71]:
coherence_model_lda.get_coherence()

0.666448795060286

In [74]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

# LDAvis_prepared
pprint(lda_model.print_topics())

[(0,
  '0.039*"pandemia" + 0.027*"queimadas" + 0.022*"bolsonaro" + 0.022*"pantanal" + 0.022*"brasil" + 0.019*"covas" + 0.017*"lava_jato" + 0.016*"mega_sena" + 0.013*"projeto" + 0.012*"governo_bolsonaro"'),
 (1,
  '0.049*"covid" + 0.034*"bolsonaro" + 0.033*"paulo" + 0.017*"sp" + 0.015*"secretario" + 0.015*"doria" + 0.015*"rio" + 0.015*"amazonia" + 0.015*"o" + 0.015*"ano"'),
 (2,
  '0.041*"brasil" + 0.039*"eua" + 0.023*"milhoes" + 0.022*"mulher" + 0.019*"policia" + 0.017*"pais" + 0.014*"analise" + 0.011*"cafe_mercado" + 0.011*"critica" + 0.010*"anos"'),
 (3,
  '0.056*"opiniao" + 0.023*"rio" + 0.020*"ministro" + 0.019*"preso" + 0.019*"bolsonaro" + 0.018*"candidatos" + 0.018*"acao" + 0.015*"rj" + 0.015*"jovem" + 0.015*"dia"'),
 (4,
  '0.035*"onu" + 0.021*"eleicoes" + 0.019*"pessoas" + 0.018*"bolsonaro" + 0.018*"pede" + 0.017*"pf" + 0.017*"durante" + 0.015*"boulos" + 0.014*"a" + 0.013*"quase"'),
 (5,
  '0.047*"governo" + 0.023*"bolsonaro" + 0.022*"homem" + 0.020*"witzel" + 0.019*"russomanno

In [77]:
lda_model.print_topics()[0][1]

'0.039*"pandemia" + 0.027*"queimadas" + 0.022*"bolsonaro" + 0.022*"pantanal" + 0.022*"brasil" + 0.019*"covas" + 0.017*"lava_jato" + 0.016*"mega_sena" + 0.013*"projeto" + 0.012*"governo_bolsonaro"'

In [89]:
for i, row in enumerate(lda_model[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    print(row)


[(3, 0.5596915), (6, 0.1977773), (1, 0.0788656), (0, 0.032733116), (2, 0.032733116), (4, 0.032733116), (5, 0.032733116), (7, 0.032733116)]
[(6, 0.74238676), (1, 0.03803722), (0, 0.036596004), (2, 0.036596004), (3, 0.036596004), (4, 0.036596004), (5, 0.036596004), (7, 0.036596004)]
[(5, 0.5594885), (6, 0.13822955), (4, 0.13818593), (1, 0.033250086), (0, 0.032711484), (2, 0.032711484), (3, 0.032711484), (7, 0.032711484)]
[(7, 0.50619215), (4, 0.31598175), (5, 0.029842226), (0, 0.029596783), (1, 0.029596783), (2, 0.029596783), (3, 0.029596783), (6, 0.029596783)]
[(6, 0.5053874), (7, 0.26436934), (3, 0.1050217), (4, 0.025797406), (0, 0.024856042), (1, 0.024856042), (2, 0.024856042), (5, 0.024856042)]
[(6, 0.62577236), (2, 0.15453906), (5, 0.036788892), (0, 0.036579944), (1, 0.036579944), (3, 0.036579944), (4, 0.036579944), (7, 0.036579944)]
[(2, 0.44143674), (0, 0.15039448), (4, 0.1383387), (1, 0.13825177), (3, 0.033355054), (5, 0.032741092), (6, 0.032741092), (7, 0.032741092)]
[(4, 0.7440